# 1. SF antimicrobial resistance dataset exploration

## 1.1 Documentation

1. [Exploring data with Pandas](https://realpython.com/pandas-python-explore-dataset/)

## 1.2 Data
1. [Antibiotic resistance](https://atlas.ecdc.europa.eu/public/index.aspx?Dataset=27&HealthTopic=4 )
2. [Antibiotic consuming](https://data.europa.eu/euodp/en/data/dataset/antimicrobial-consumption-data/resource/f38b4302-077f-4397-889e-47593b78436c)

### 1.2.1 Total antibiotic resistance data (ECDC)

In [2]:
import pandas as pd

df = pd.read_csv('ECDC_surveillance_data_Antimicrobial_resistance FULL.csv').drop(labels=['HealthTopic','TxtValue'],axis=1)
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 2)

df['Time'] =  pd.to_datetime(df['Time'],format='%Y')
new = df['Population'].str.split("|", n = 1, expand = True)

df['Bacteria'] = new[0]
df['AntimicrobialGroup'] = new[1]

df = df.drop(labels='Population',axis=1)

df.head(10)

df.to_csv('ECDC_clean.csv')

In [5]:
print(df.isnull().sum())


Indicator             0
Unit                  0
Time                  0
RegionCode            0
RegionName            0
NumValue              0
Bacteria              0
AntimicrobialGroup    0
dtype: int64


In [51]:
print('unique values col Bacteria :',df['Bacteria'].nunique())
print('unique values col AntimicrobialGroup :',df['AntimicrobialGroup'].nunique())
print('unique values col Indicator :',df['Indicator'].nunique())
print('unique values col RegionCode :',df['RegionCode'].nunique())

unique values col Bacteria : 8
unique values col AntimicrobialGroup : 16
unique values col Indicator : 8
unique values col RegionCode : 30


### 1.2.2 Total antibiotic consumption, in community and hospital datasets

In [1]:
import pandas as pd
xls = pd.ExcelFile('ESAC_total.xlsx')
esac1 = pd.read_excel(xls, 'Sheet1')
esac2 = pd.read_excel(xls, 'Sheet2')

esac1 = esac1.fillna(0)
esac2 = esac2.fillna(0)

esac1.to_csv('total_esac_community.csv')
esac2.to_csv('total_esac_hospital.csv')

### 1.2.3 Antibiotic consumption per group, in community dataset

In [75]:
import pandas as pd

def concatDataFrames(xls, fileName):
    sheetNames = xls.sheet_names
    complete_df = pd.DataFrame(columns=['Country', 'Year', 'Value','AntibioticGroup'])
    for sheet in sheetNames:
        # read each sheet and fill na with 0
        df = pd.read_excel(xls, sheet).fillna(0)
        # unpivot table
        df = df.melt(id_vars=['Country'])
        # add antibiotic group column
        df['AntibioticGroup'] = sheet
        # rename columns
        df = df.rename(columns = {'variable': 'Year','value':'Value'}, inplace = False)
        # concat the tables
        complete_df = complete_df.append(df, ignore_index=True)
        
    complete_df.to_csv(fileName)


xls1 = pd.ExcelFile('ESAC_antimicrobial_groups_community.xlsx')
concatDataFrames(xls1,'esac_antimicrobial_groups_community.csv')


### 1.2.4 Antibiotic consumption per group, in hospitals dataset

In [76]:
xls2 = pd.ExcelFile('ESAC_antimicrobial_groups_hospital.xlsx')
concatDataFrames(xls2,'esac_antimicrobial_groups_hospital.csv')